In [ ]:
from pathlib import Path
from typing import Annotated

import pytask
from pytask import ExitCode
from pytask import Product
from pytask import task

In [ ]:
# Create repeated tasks with the same function name
tasks = []
for i in range(3):

    def create_data(value: int, produces: Annotated[Path, Product]):
        """Generate data based on a value."""
        produces.write_text(str(value))

    t = task(
        kwargs={"value": i * 10, "produces": Path(f"data_{i}.txt").resolve()},
    )(create_data)
    tasks.append(t)

In [ ]:
# Test that all tasks execute correctly
session = pytask.build(tasks=tasks)
assert session.exit_code == ExitCode.OK
assert len(session.tasks) == 3, f"Expected 3 tasks, got {len(session.tasks)}"
assert len(session.execution_reports) == 3, (
    f"Expected 3 execution reports, got {len(session.execution_reports)}"
)

# Verify each file was created with the correct content
assert Path("data_0.txt").read_text() == "0"
assert Path("data_1.txt").read_text() == "10"
assert Path("data_2.txt").read_text() == "20"

# Clean up
Path("data_0.txt").unlink()
Path("data_1.txt").unlink()
Path("data_2.txt").unlink()